In [34]:
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import os,sys
from glob import glob
from scipy.optimize import fmin
from scipy.optimize import curve_fit
from scipy.signal import find_peaks
from scipy import interpolate
import time
from multiprocessing import Process, Array, Lock

In [2]:
basedir = "/Users/laote/sdsu/Research/binary/tess_alldata"
curvedir = "/Users/laote/sdsu/Research/binary/tess_alldata/Curves"

# binary_list = np.genfromtxt("classification.txt",dtype=str)
# t_binary_dir = []
# k_binary_dir = []
# whole_binary_dir = []

# for i in range(len(binary_list)):
#     whole_binary_dir.append(curvedir + "/%s_LC.txt" %binary_list[i,0])
#     if binary_list[i,0][0] == 'k':
#         k_binary_dir.append(curvedir + "/%s_LC.txt" %binary_list[i,0])
#     elif binary_list[i,0][0] == 't':
#         t_binary_dir.append(curvedir + "/%s_LC.txt" %binary_list[i,0])
#     else:
#         print("???")

In [3]:
def find_all_byname():
    binary_list = np.genfromtxt("binary.txt",dtype=str)
    binary_dir = []
    for b in binary_list:
        search = '/Users/laote/sdsu/Research/binary/curve_bin/Curve_binary_320000sort/%s_*_LC.txt'%b
        look_dir = glob(search)
        for i in range(len(look_dir)):
            binary_dir.append(look_dir[i])
    return binary_dir

In [33]:
def see_period(datadir):
    data = np.genfromtxt(datadir,names="time,flux,et,ef")
    use = (data['ef'] == 0)

    time_diff = data['time'][use][1:] - data['time'][use][:-1]
    error_point = np.argmax(time_diff)
    
    Time = np.append(data['time'][use][40:error_point-50],data['time'][use][error_point+50:-40])
    flux = np.append(data['flux'][use][40:error_point-50],data['flux'][use][error_point+50:-40])

    med = np.median(flux) 
    std = np.std(flux)
    
    plt.figure(figsize=(10,5))
    plt.plot(Time,flux,'c')
    plt.plot(Time,flux,'r.')
    plt.show()

In [5]:
def light_curve(lc_dir,look=False):
    data = np.genfromtxt(lc_dir,names="time,flux,et,ef")
    good = (data['ef'] == 0)
    time_diff = data['time'][good][1:] - data['time'][good][:-1]
    
    if (np.max(time_diff) < 2) & (np.sum(time_diff > 0.7) <2):
        error_point = np.argmax(time_diff)

        time = np.append(data['time'][good][30:error_point-50],data['time'][good][error_point+50:-30])
        flux = np.append(data['flux'][good][30:error_point-50],data['flux'][good][error_point+50:-30])

        norm_flux = (flux - np.median(flux))/(np.max(flux) - np.min(flux))
    
        if look == True:
            plt.figure(figsize=(10,5))
            plt.plot(time,norm_flux,'c')
            plt.xlabel('time');plt.ylabel('flux')
            plt.show()
        else:
            None
    
        return time,norm_flux

    else:
        return "bad_data","bad_data"


def mulit_light_curve(lc_dir,look=False):
    time = []; flux = []
    size = 0
    for i in range(10):
        test_dir = lc_dir[:-9] + "0%s" %i + "_LC.txt"
        if os.path.exists(test_dir) == True:
            t,f = light_curve(test_dir)
            if t == 'bad_data':
                None
            else:
                time.extend(t)
                flux.extend(f)
                size += 1
        else:
            None

    for i in range(10):
        test_dir = lc_dir[:-9] + "1%s" %i + "_LC.txt"
        if os.path.exists(test_dir) == True:
            t,f = light_curve(test_dir)
            if t == 'bad_data':
                None
            else:
                time.extend(t)
                flux.extend(f)
                size += 1
        else:
            None
    if size > 2:
        if look == True:
            plt.figure(figsize=(5*size,5))
            plt.plot(time,flux,'c')
            plt.plot(time,flux,'r.')
            plt.xlabel('time');plt.ylabel('flux')
            plt.savefig("/Users/laote/sdsu/Research/binary/curve_bin/Binary_img/%s.png"%lc_dir[-28:-14])
        else:
            None
    else:
        None

    return time, flux

In [6]:
# whichone = 832
# print(t_binary_dir[whichone])
# time,flux = mulit_light_curve(t_binary_dir[whichone],look=True)
# time,flux = light_curve(t_binary_dir[whichone],look=True)

In [7]:
def success_mulit_light_curve(lc_dir,all_dir,look=False):
    time = []; flux = []
    size = 0
    for i in range(10):
        test_dir = lc_dir[:-9] + "0%s" %i + "_LC.txt"
        if (test_dir in all_dir) == True:
            t,f = light_curve(test_dir)
            if t == 'bad_data':
                None
            else:
                time.extend(t)
                flux.extend(f)
                size += 1
        else:
            None

    for i in range(10):
        test_dir = lc_dir[:-9] + "1%s" %i + "_LC.txt"
        if (test_dir in all_dir) == True:
            t,f = light_curve(test_dir)
            if t == 'bad_data':
                None
            else:
                time.extend(t)
                flux.extend(f)
                size += 1
        else:
            None
            
    if look == True:
        plt.figure(figsize=(5*size,5))
        plt.plot(time,flux,'c.')
        plt.xlabel('time');plt.ylabel('flux')
        plt.show()
    else:
        None

    return time, flux

In [8]:
# time,flux = success_mulit_light_curve(t_binary_dir[whichone],whole_binary_dir,look=True)

In [9]:
def Phase_cycle(t,P,T_0):
    T_diff = np.array(t) - T_0
    I = np.floor(T_diff/P)
    phi = (T_diff/P) - I
    return phi, I #phase and cycle

def dc_func(guess_P,lc_dir,all_dir,look=False):
    time,norm_flux = success_mulit_light_curve(lc_dir,all_dir)
    phi, I = Phase_cycle(time, guess_P, time[0])
    phase_flux = np.stack((phi,norm_flux),axis=-1)
    phase_flux = np.array(sorted(phase_flux,key=lambda x:x[0]))
    phase = phase_flux[:,0]
    flux = phase_flux[:,1]

    if look == True:
        plt.figure(figsize=(10,7))
        plt.plot(phase,flux,'c.')
        plt.xlabel('phase');plt.ylabel('flux')
        plt.show()        
    
    S = np.fabs(flux[-1] - flux[0]) + np.sum(np.fabs(flux[1:] - flux[:-1]))
    return S/47.-2.

def sing_dc_func(guess_P,lc_dir,look=False):
    time,norm_flux = light_curve(lc_dir)
    phi, I = Phase_cycle(time, guess_P, time[0])
    phase_flux = np.stack((phi,norm_flux),axis=-1)
    phase_flux = np.array(sorted(phase_flux,key=lambda x:x[0]))
    phase = phase_flux[:,0]
    flux = phase_flux[:,1]

    if look == True:
        plt.figure(figsize=(10,7))
        plt.plot(phase,flux,'c-')
        plt.plot(phase,flux,'r.')
        plt.xlabel('phase');plt.ylabel('flux')
        plt.savefig('/Users/laote/sdsu/Research/binary/curve_bin/Binary_period/%s_P_%.3f.png'\
                    %(lc_dir[-28:-7],guess_P))
#         plt.show()        
    
    S = np.fabs(flux[-1] - flux[0]) + np.sum(np.fabs(flux[1:] - flux[:-1]))
    return S/47.-2.

In [10]:
binary_dir = find_all_byname()
binary_dir[0][-28:-7]
binary_dir[0][-28:-14]

'tic00349575009'

In [10]:
def find_period_SL(datadir,bins=10000):
#     start = time.time()
    output_dir = '/Users/laote/sdsu/Research/binary/curve_bin/Binary_period'
    dc = []
    guess_p = np.linspace(1,15,bins)
    for gp in guess_p:
        dc.append(sing_dc_func(gp,datadir))
    sing_dc_func(guess_p[np.argmin(dc)],datadir,look=True)
    
#     print(time.time()-start)
    return guess_p[np.argmin(dc)]

In [12]:
# procs = []

# for i in range(100):
#     p = Process(target = find_period_SL, args = (binary_dir[201+i],10000))
#     procs.append(p)
#     p.start()

# for p in procs:
#     p.join()

# for i in range(800):
#     find_period_SL(binary_dir[200+i])

In [13]:
binary_name =  glob("/Users/laote/sdsu/Research/binary/curve_bin/Binary_period/*.png")

In [12]:
def comb_peri(dir_name):
    binary_dict = dict()

    for d in dir_name:
        ta = d[58:72]
        tb = float(d[82:-4])
        if ta in binary_dict.keys():
            binary_dict[ta].append(tb)
        else:
            binary_dict[ta] = [tb]
            
    return binary_dict

all_period = comb_peri(binary_name)

final_period = dict()
for a in all_period.keys():
    p_ls = all_period['%s'%a]
    minimum = np.min(p_ls)
    total = [minimum]
    for p in p_ls:
        if p/minimum < 10:
            if str(p/minimum)[2] == '9':
                new_p = p/(int(p/minimum)+1)
                total.append(new_p)
            elif (str(p/minimum)[2] == '0') | (str(p/minimum)[2] == '1'):
                new_p = p/(int(p/minimum))
                total.append(new_p)
            
        
    final_p = np.mean(total)

    final_period[a] = final_p


In [38]:
f = open("period_v1.txt","w")
for k in final_period.keys():
    f.write('%s  %.4f\n'%(k,final_period[k]) )
f.close()

final_period



{'tic00030719539': 9.313,
 'tic00167718833': 1.6333703703703704,
 'tic00271905985': 10.919,
 'tic00277026877': 2.1727777777777777,
 'tic00350335627': 2.3889166666666664,
 'tic00167719767': 1.2111,
 'tic00279090253': 1.4983125,
 'tic00350335398': 6.074150000000001,
 'tic00149971074': 8.646875000000001,
 'tic00029924983': 10.958333333333334,
 'tic00141622065': 2.9279166666666665,
 'tic00141768070': 2.785558333333334,
 'tic00055404385': 2.952444444444444,
 'tic00382268993': 1.4216555555555557,
 'tic00055602946': 4.9385666666666665,
 'tic00179203170': 2.674822222222222,
 'tic00141709240': 1.4711363636363637,
 'tic00404970655': 6.9562,
 'tic00141871739': 4.592333333333333,
 'tic00055497729': 1.246388888888889,
 'tic00278683875': 7.75,
 'tic00300653838': 1.042,
 'tic00381979901': 3.2535625,
 'tic00141978095': 13.0244,
 'tic00055650498': 5.604666666666667,
 'tic00300604770': 3.454092592592593,
 'tic00389572057': 12.088125000000002,
 'tic00349519544': 9.554750000000002,
 'tic00149971575': 9.25

In [16]:
# binaryseedir = '/Users/laote/sdsu/Research/binary/curve_bin/Curve_binary_320000sort/'
# look_dir = glob("%s/*.txt" %binaryseedir)

In [17]:
# a = 0
# b = len(look_dir)
# for ldir in look_dir:
# #     print(ldir)
# #     see_period(ldir)
#     a+=1
#     time,flux = mulit_light_curve(ldir,look=True)
#     if a % 5 == 0:
#         output = a/b*100
#         sys.stdout.write('\r loading... %.4f %%'%output)
#         sys.stdout.flush()

# #     time,flux = light_curve(ldir,look=True)

In [15]:
# tic00142049289_SEC002
# tic00030313070_SEC005
# tic00350622185_SEC006
# tic00141870501_SEC012
# tic00031107108_SEC005

In [16]:
# datadir = '/Users/laote/sdsu/Research/binary/curve_bin/Binary_img/'
# aaa =  glob("%s/*.png" %datadir)

# f = open('binary.txt','w')
# for a in aaa:
#     f.write(a[-18:-4]+"\n")
# f.close()

In [31]:
# sort32 =  glob("/Users/laote/sdsu/Research/binary/curve_bin/Curve_binary/*.txt")

# recordin = []
# for aa in sort32:
#     serdir = "/home/zlin/research/Curve_binary/"+aa[-28:-14]
#     if serdir not in recordin:
#         recordin.append(serdir)
        

# f = open('km_binary_32list.txt','w')
# for aaa in recordin:
#     f.write(aaa+"*.txt"+"\n")
# f.close()


In [ ]:
see_period()